In [1]:
import torch 
import torch_geometric
import torch.nn.functional as F

In [ ]:
x=torch.arange(12).reshape(3,4).float()


tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [6]:
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [11]:
x[0]

tensor([0., 1., 2., 3.])

In [13]:
F.normalize(x,dim=1)


tensor([[0.0000, 0.2673, 0.5345, 0.8018],
        [0.3563, 0.4454, 0.5345, 0.6236],
        [0.4182, 0.4704, 0.5227, 0.5750]])

In [16]:
torch.matmul(F.normalize(x,dim=0)[0],F.normalize(x,dim=0)[0])

tensor(0.0882)

In [2]:
num_nodes = 4
embedding_dim = 8
top_k = 2
directed = True

# Learnable time series embeddings
node_embeddings = torch.nn.Parameter(torch.Tensor(num_nodes, embedding_dim))


In [ ]:
from torch_geometric.nn import GAT

m=GAT(2,4,1,dropout=0.5,add_self_loops=True,negative_slope=0.2,v2=True)   

In [ ]:
tor

In [4]:
m(torch.rand(num_nodes,2)
  ,edge_index=torch.tensor([[0, 1, 2, 3], [1, 2, 3, 0]])
#   ,return_attention_weights =True
  )#,edge_attr=torch.rand(4,embedding_dim))  

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.8767,  1.5158,  0.6576, -0.2324],
        [-0.4523,  0.8355,  0.3821, -0.2194],
        [-0.5404,  0.9982,  0.4565, -0.2621]], grad_fn=<AddBackward0>)

In [19]:
torch.nn.init.xavier_uniform_(node_embeddings)

Parameter containing:
tensor([[ 0.2892,  0.2088,  0.5510, -0.5835,  0.0499, -0.3093, -0.2231,  0.4342],
        [ 0.2747, -0.6460,  0.6218,  0.0903,  0.1982, -0.1554, -0.0335, -0.0292],
        [ 0.2904,  0.5400, -0.2818,  0.3414,  0.5780,  0.5268,  0.6351,  0.1526],
        [ 0.4842, -0.2012,  0.2333,  0.0155, -0.2962, -0.2112,  0.6932,  0.1884]],
       requires_grad=True)

In [14]:
node_embeddings

Parameter containing:
tensor([[1.6909e+07, 1.6381e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]], requires_grad=True)

In [21]:
normalized_embeddings = F.normalize(node_embeddings, p=2, dim=1)
node_embeddings


Parameter containing:
tensor([[ 0.2892,  0.2088,  0.5510, -0.5835,  0.0499, -0.3093, -0.2231,  0.4342],
        [ 0.2747, -0.6460,  0.6218,  0.0903,  0.1982, -0.1554, -0.0335, -0.0292],
        [ 0.2904,  0.5400, -0.2818,  0.3414,  0.5780,  0.5268,  0.6351,  0.1526],
        [ 0.4842, -0.2012,  0.2333,  0.0155, -0.2962, -0.2112,  0.6932,  0.1884]],
       requires_grad=True)

In [22]:
# Compute cosine similarity matrix
sim_matrix = torch.mm(normalized_embeddings, normalized_embeddings.t())
sim_matrix


tensor([[ 1.0000,  0.2796, -0.2750,  0.1875],
        [ 0.2796,  1.0000, -0.3282,  0.3675],
        [-0.2750, -0.3282,  1.0000,  0.1259],
        [ 0.1875,  0.3675,  0.1259,  1.0000]], grad_fn=<MmBackward0>)

In [23]:
# Ensure self-similarity is not selected
sim_matrix.fill_diagonal_(-1)  # Set diagonal to -1 to exclude self-connections

# Get top-k neighbors for each node
topk_values, topk_indices = torch.topk(sim_matrix, k=top_k, dim=1)
topk_values, topk_indices


(tensor([[ 0.2796,  0.1875],
         [ 0.3675,  0.2796],
         [ 0.1259, -0.2750],
         [ 0.3675,  0.1875]], grad_fn=<TopkBackward0>),
 tensor([[1, 3],
         [3, 0],
         [3, 0],
         [1, 0]]))

In [24]:
# Create binary adjacency mask
adj_mask = torch.zeros_like(sim_matrix)
adj_mask.scatter_(1, topk_indices, 1.0)
adj_mask


tensor([[0., 1., 0., 1.],
        [1., 0., 0., 1.],
        [1., 0., 0., 1.],
        [1., 1., 0., 0.]])

In [ ]:
# For undirected graphs, ensure symmetric connections
if not directed:
    adj_mask = torch.max(adj_mask, adj_mask.t())
    # Remove self-loops
    adj_mask.fill_diagonal_(0)

In [27]:
torch.nn.Parameter(torch.empty(1, 2, 5))

Parameter containing:
tensor([[[-1.6988e+32,  1.1280e-42,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]],
       requires_grad=True)

In [ ]:
torch.nn.Parameter()

TypeError: Parameter.__new__() takes from 1 to 3 positional arguments but 4 were given

In [25]:
node_embeddings.shape

torch.Size([4, 8])

In [26]:
x=torch.rand(4,3)
x

tensor([[0.9361, 0.0897, 0.6139],
        [0.5780, 0.2410, 0.9744],
        [0.1290, 0.1885, 0.6918],
        [0.7335, 0.6535, 0.1109]])

In [24]:
import torch_geometric.utils


adj=torch.ones(4,4)
adj,f=torch_geometric.utils.dense_to_sparse(adj)
adj,f=torch_geometric.utils.remove_self_loops(adj)

In [25]:
model = torch_geometric.nn.GAT(num_layers=1,hidden_channels=6,in_channels=-1,out_channels=2)

In [32]:
model(x,adj)

tensor([[-0.9533,  0.6493],
        [-0.9533,  0.6493],
        [-0.9533,  0.6493],
        [-0.9533,  0.6493]], grad_fn=<AddBackward0>)